# 24. Ch 03. Preprocessing - 17. 미니배치 만들기

In [ ]:
미니배치, 토치 텍스트
모델에 넣기 위한 마지막 변환 과정
Split Train/Valid/Test -> Read text& Build dictionary(index mapping) -> Sort by length
-> Conver to integer -> Batchify and pad -> Shuffle and feed

<Read text& Build dictionary>
-빈도 순으로 단어 사전 정렬
-필요에 따라 min_count보다 작은 빈도를 갖는 단어는 제외
또는 max_vocab에 따라 빈도순으로 어휘를 제외하기도 함
-필요에 따라 특수 토큰도 어휘 사전에 포함
<BOS>, <EOS>, <UNK>, <PAD> 등
Begin of Sentence, End of Sentence, Unknown Token, Padding

나는 뷁뛣에 가서 아침 식사를 했어요.
-> 나 는 뷁뛣 에 가 서 아침 식사 를 했 어 요 .
-> <BOS> 나 는 <UNK> 에 가 서 아침 식사 를 했 어 요 . <EOS>

<Chunking and Padding>
-미니배치의 형태
(batch_size, length, |V|)
|V|: 보카뷰러리 사이즈, 하나가 원 핫 벡터가 됨
미니배치가 이러한 형태의 텐서가 된다
원핫벡터가 길이 만큼 있어서 문장이 이뤄지고 문장들이 모여서 미니배치가 되는 것

-원핫 벡터를 다 저장할 필요가 없음
인덱스 값만 가지고 있으면 된다.
(batch_size, length, 1) =  (batch_size, length)

-Sequence 차원의 크기는 미니배치 내의 가장 긴 문장에 의해 결정됨
비어 있는 부분은 뭔가로 채워줘야 한다.
각 샘플별 모자라는 부분은 패딩으로 대체 , 따라서 <PAD> 토큰이 필요.
패드에 대한 인덱스도 생각해줘야 한다는 것.
나중에 파이토치의 PackedSequence를 활용할 경우, <PAD> 생략 가능
패드 토큰은 여기에 토큰이 있긴 한데 비어 있는 토큰이다고 표시해놓는 것.

-패딩해주면 문제점? 재수없어서 처음에는 100단어 문장, 다음에는 2단어 문장이라면
98타임스텝이 패드로 채워진다.
-> 길이에 맞는 미니배치 필요

<Increase Training Efficiency>
1)sort by sequence length
2)Get chunks with similar length of sequences.(chunking)
청킹 이후 미니배치 순서를 셔플링해주면 된다.
훨씬 효율적으로 학습 진행 가능

<효율적 학습이 가능한 미니배치 만들기>
1)코퍼스의 각 문장들을 길이에 따라 정렬
2)각 토큰들을 사전을 활용해 str->index맵핑
3)미니배치 크기대로 청킹
4)각 미니배치 별 텐서 구성 및 패딩
5)학습 시 미니배치 셔플링해 iterative하게 반환

이걸 편리하게 해주는 라이브러리가 토치텍스트

<Torch Text>
앞서 소개한 작업들을 수행해주는 파이토치 공식 텍스트 로딩용 라이브러리
현재버전 0.5.1

<Define Task: what you want>
-f(text) = class
Text classification
-f(text) = word
Language Modeling
-f(text) = text
Machine Translation

텍스트, 클래스의 텐서 모양
|text| = (batch_size, length, |V|)
|class| = (batch_size, length, |C|)

<Step 1: Define Fields> 
필드 정의

self.label = data.Field(sequential=False,
			use_vocab=True,
			unk_token=None
			)
self.text = data.Field(use_vocab=True,
			batch_first=True,
			include_lengths=False,
			eos_token=‘<EOS>’ if use_eos else None

<Step2: Define Dataset with Fields>
train, valid = data.TabularDataset.splits(path=‘’.
					train=train_fn,
					validation=valid_fn,
					format=‘tsv’,
					fields=[(‘label’, self.label),
						(‘text’, self.text)
						]
					)

<Step3: Get DataLoaders from Datasets>
self.train_iter, self.valid_iter = data.BucketIterator.splits((train, valid),
							batch_size=batch_size,
							device=‘cuda:%d’ % device if device >= 0 else ‘cpu’,
							shuffle=shuffle,
							sort_key=lambda x: len(x.text),
							sort_within_batch=True
							)
self.label.build_vocab(train)
self.text.build_vocab(train, max_size=max_vocab, min_freq=min_freq)

테스트셋 셔플링 안 해준다, 트레이닝, 밸리데이션 셋에만

<Example Usage:>
-Vocabulary(or Class) size
vocab_size = len(dataset.text.vocab)
n_classes = len(dataset.label.vocab)
print(‘|vocab| =’, vocab_size, ‘|classes| =’, n_classes)

-Feeding with Ignite
trainer.run(train_loader, max_epochs=self.config.n_epochs)

# 25. Ch 03. Preprocessing - 18. 실습 TorchText

In [ ]:
-문서 라인 수
wc -l ./review.sorted.uniq.refined.tok.*

-셔플링
gshuf < review.sorted.uniq.refined.tok.tsv > review.sorted.uniq.refined.tok.shuf.tsv

-train 셋
head -n 212680 ./review.sorted.uniq.refined.tok.shuf.tsv > review.sorted.uniq.refined.tok.shuf.train.tsv

-테스트 셋
tail -n 90000 ./review.sorted.uniq.refined.tok.shuf.tsv > review.sorted.uniq.refined.tok.shuf.test.tsv

# 26. Ch 03. Preprocessing - 19. 정리하며

In [ ]:
<NLP 프로젝트 워크플로>
-문제 정의 
단계를 나누고 simplify
x와 y를 정의
-데이터 수집 
문제 정의에 따른 수집
필요에 따라 레이블링
-데이터 전처리 및 분석
형태를 가공
필요에 따라 EDA 수행
-알고리즘 적용
가설을 세우고 구현/적용
-평가
실험 설계
테스트셋 구성
 -배포
RESTful API를 통한 배포
상황에 따라 유지/보수

<전처리 워크플로>
-데이터(코퍼스) 수집
구입, 외주
크롤링을 통한 수집
-정제
태스크에 따른 노이즈 제거(이 노이즈가 의미가 있을까를 고민)
인코딩 변환(utf-8로 통일 추천, 다국어 다루기 위해선)
-레이블링(Optional)
문장마다 또는 단어마다 레이블링 수행
-토크나이제이션
형태소 분석기를 활용해 분절 수행
한국어는 필수. 교착어이기 때문에. 접사를 분리하지 않으면 희소성이 커진다.
-서브워드 세그멘테이션(Optional)
단어보다 더 작은 의미 단위
추가 분절 수행
-Batchify
사전 생성 및 word2index 맵핑 수행
효율화를 위한 전/후처리
torchText를 활용해 쉽게 처리할 수 있다.

<Service Pipeline>
-정제(regex, encoding)
학습데이터와 같은 방식의 정제 수행
-토크나이제이션(mecab)
학습 데이터와 같은 방식의 분절 수행
-서브워드 세그멘테이션(BPE)
학습 데이터로부터 얻은 '모델'을 활용해 똑같은 분절 수행
코퍼스마다 모델이 새롭게 하나씩 생성(BPE를 학습하는 것이기 때문에)
-Batchify(torchtext)
학습 데이터로부터 얻은 사전에 따른 word2index 맵핑 수행
-Prediction
모델에 넣고 추론 수행
필요에 따라 serach 수행(자연어 생성)
-Detokenization(Optional)
사람이 읽을 수 있는 형태로 변환(index2word)
분절 복원

보통은 API 서버를 만들텐데 이 서버 안에 위의 파이프라인이 다 들어가 있어야 함

<요약>
-정제
태스크와 언어 및 도메인에 따른 특성
	풀고자 하는 문제의 특성에 따라 전처리 전략이 다름
끝이 없는 과정
	노력과 품질 사이의 트레이드 오프
	sweet spot을 찾아야 함
-분절
한국어의 경우 띄어쓰기 노말라이제이션을 위해 형태소 분석기 활용이 필요
서브워드 세그멘테이션을 통해 좀 더 잘게 분절할 수 있음
-모두 비슷한 알고리즘을 사용하고 있으므로, 결국 데이터의 양과 품질이 좌우함
따라서 전처리 과정을 경시해서는 안 됨